In [1]:
from sklearn.feature_extraction.text import TfidfTransformer, CountVectorizer
from nltk.stem.isri import ISRIStemmer
from nltk.corpus import stopwords
from math import log10
import networkx as nx
import re

stemmer = ISRIStemmer()

In [2]:
# # download the stopwords if you dont have them
# import nltk
# nltk.download()

In [3]:
def sentence_stem(sentence):
    stemmed_sentence = ''
    arr = []
    for word in sentence.strip().split():
        if word not in (stopwords.words('arabic')):
            arr.append(stemmer.stem(word))
    stemmed_sentence = ' '.join(arr)
    return stemmed_sentence

def similarity(s1, s2):
    stemsen1 = sentence_stem(s1)
    stemsen2 = sentence_stem(s2)
    wc1 = len(stemsen1.split())
    wc2 = len(stemsen2.split())
    common_wc = len(set(stemsen1.split()) & set(stemsen2.split()))
    if wc1 ==0 or wc2 ==0:
        return 0
    if (log10(wc1)+log10(wc2)) == 0:
        return 0
    return common_wc/(log10(wc1)+log10(wc2))

In [4]:
def tldr_matrix(article):
    article = re.sub(r'[^\w\s\.]', ' ', article)
    sentences = article.split('.')
    stemmed_sentences = []
    for sentence in sentences:
        stemmed_sentences.append(sentence_stem(sentence))
    
    bagofwords_matrix = CountVectorizer().fit_transform(stemmed_sentences)
    # normalize with TF-IDF
    bagofwords_matrix = TfidfTransformer().fit_transform(bagofwords_matrix)
    
    # mirror the matrix onto itself to get the similarity edges between sentences
    similarity_matrix = bagofwords_matrix * bagofwords_matrix.T
    
    similarity_graph = nx.from_scipy_sparse_matrix(similarity_matrix)
    scores = nx.pagerank(similarity_graph)
    
    return sorted(((scores[i],s) for i,s in enumerate(sentences)), reverse=True)


# this method is more expensive but is more faithful to the paper
def tldr_graph(article):
    article = re.sub(r'[^\w\s\.]', ' ', article)
    sentences = article.split('.')
    graph = nx.Graph()
    graph.add_nodes_from(sentences)
    for sen1 in graph.nodes():
        for sen2 in graph.nodes():
            edge_w = similarity(sen1, sen2)
            if edge_w !=0:
                graph.add_edge(sen1, sen2, weight=edge_w)
    
    scores = nx.pagerank(graph)
    return sorted([(value, key) for key, value in scores.items()], reverse = True)

In [5]:
def summarize(article, method='m', factor = 2):
    if method == 'm':
        smry = tldr_matrix(article)
    elif method == 'g':
        smry = tldr_graph(article)
    else:
        raise ValueError('invalid method')
    return '\n'.join([_[1] for _ in smry[:factor]])

In [8]:
# TODO: return sentences in their original order in the article?
f = open('article.txt', encoding='utf-8')
article = f.read()

smry1 = summarize(article, method='g')
print('graph method')
print(smry1)
smry2 = summarize(article)
print('matrix method')
print(smry2)

graph method
 بما أننا في بداية العام الدراسي الجديد أسعدني وكل مهتم أن التعليم يفكرون في إضافة مادة تعنى بالادخار والتخطيط المالي وهذه خطوة إذا تمت ستساهم في نقل الادخار من ثقافة إلى تطبيق مع مرور الوقت وزيادة التوعية  في منظوري الشخصي لا شيء يثبت اهتمام الناس بالادخار من القبول الذي وجده المقال الماضي من قراء المقالات وكذلك تعدد المقالات التي تناولت الادخار في الفترة الماضية ولست أولهم ولا أنسى الدورات التدريبية التي حالت مدن المملكة وشهدت حضور المهتمين  إذا الادخار أصبح حالة تلفت وتستقطب الناس إليها

لكي لا نعيد التجارب السابقة في الادخار والحرية المالية وحتى لا نبدأ من الصفر مرة أخرى يجب علينا فهم كيف يتصرف الأفراد بدخولهم   وماهي الكماليات فعلا التي تكلف الناس عدم القدرة على الادخار   وهل الادخار يأتي بحفظ المال  أدخر أولا  أم بالعمل أكثر  تعزيز الدخل  ظروف اليوم تغير من طبيعة قدرة الفرد الحصول على دخل إضافي ليس كما كان سابقا
matrix method

لكي لا نعيد التجارب السابقة في الادخار والحرية المالية وحتى لا نبدأ من الصفر مرة أخرى يجب علينا فهم كيف يتصرف الأفراد بدخولهم   وماهي الكماليا